# VGG16을 사용하여 아이돌 이미지 분류
- 이미지는 Kaggle에서 다운로드 가능(https://www.kaggle.com/datasets/vkehfdl1/kidf-kpop-idol-dataset-female?resource=download&select=kid_f_train.csv)


In [1]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

from sklearn.preprocessing import LabelEncoder

In [2]:
url = './dataset'

csv_url_train = url + '/kid_f_train.csv'
csv_url_test = url + '/kid_f_test.csv'


train_labels = pd.read_csv(csv_url_train)
test_labels = pd.read_csv(csv_url_test)

In [3]:
train_labels.head(3)

,file_name,name
0,5.jpg,yujin
1,8.jpg,yujin
2,17.jpg,yujin


In [ ]:
test_labels.head(3)

,file_name,name
0,45.jpg,yujin
1,67.jpg,iu
2,85.jpg,winter


In [ ]:
# 라벨값 비율 확인
print('훈련 데이터셋 shape > ', train_labels.shape)
print('훈련 데이터셋 비율 top 5')
print((train_labels.groupby('name').size() / train_labels.shape[0] * 100).sort_values(ascending=False).head(5))

print('\n테스트 데이터셋 shape > ', test_labels.shape)
print('테스트 데이터셋 비율 top 5')
print((test_labels.groupby('name').size() / test_labels.shape[0] * 100).sort_values(ascending=False).head(5))

훈련 데이터셋 shape >  (5591, 2)
훈련 데이터셋 비율 top 5
name
lisa        22.929708
rose        22.267931
jisoo       18.297263
jennie       7.404758
kimminju     4.668217
dtype: float64

테스트 데이터셋 shape >  (300, 2)
테스트 데이터셋 비율 top 5
name
jisoo     24.000000
iu        21.333333
rose      14.000000
lisa      13.333333
jennie     5.000000
dtype: float64


# 데이터 전처리
- 훈련 데이터셋 중 lisa, rose, jisoo의 비율이 절반 이상이 넘음 => 따라서 세 명은 훈련 데이터, 학습 데이터에서 삭제
- 테스트 데이터셋에는 iu, chaeyoungdl 있지만, 테스트 데이터셋에는 없음 => 두 명도 삭제
- 텐서화 
 - 세명의 데이터를 삭제한 데이프레임을 기준으로 이미지 불러와서 텐서화
 - 이미지파일만 있는 것이 아니라 
 Thumbs.db' 파일도 있으므로 텐서로 변활할 때 에러 발생이 예상되므로 예외처리 하기


In [6]:
# 리사, 로제, 지수 제거
drop_name = ['lisa', 'rose', 'jisoo', 'iu', 'chaeyoung']

train_drop_index = train_labels[train_labels['name'].isin(drop_name)].index
test_drop_index = test_labels[test_labels['name'].isin(drop_name)].index

train_labels = train_labels.drop(train_drop_index)
test_labels = test_labels.drop(test_drop_index)

In [7]:
# 라벨 인코딩
encoder = LabelEncoder()
encoder.fit(train_labels['name'])

train_encoded = encoder.transform(train_labels['name'])
test_encoded = encoder.transform(test_labels['name'])

train_labels['name'] = train_encoded
test_labels['name'] = test_encoded

In [8]:
print("###### 세 명 삭제 후 ######")
print("train_labels 데이터 개수 : ", len(train_labels))
print("train_labels 클래스 개수 : ", len(train_labels['name'].unique()))
print('----------' * 3)
print("test_labels 데이터 개수 : ", len(test_labels))
print("test_labels클래스 개수 : ", len(test_labels['name'].unique()))

###### 세 명 삭제 후 ######
train_labels 데이터 개수 :  2041
train_labels 클래스 개수 :  52
------------------------------
test_labels 데이터 개수 :  81
test_labels클래스 개수 :  20


In [9]:
def image_to_tensor(df, dir_url):
    result_list = []

    for i in tqdm(range(df.shape[0])):
        try:
            file_name = df['file_name'].iloc[i] # 파일 이름 Ex)100.jpg
            label = df['name'].iloc[i] # 연예인 이름

            # 이미지 오픈 및 텐서화
            file_url = dir_url + '\\' + file_name
            img = Image.open(file_url)
            tf = transforms.Compose(
                [transforms.ToTensor(),
                transforms.Resize(224,224),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
            )
            
            tensor_img = tf(img)
            # 튜플 형태로 저장
            result_list.append((tensor_img, label))
        # except:
        #     print("에러 난 파일명 > ", file_name)
        except Exception as err:
            print("ERROR CODE > ", err)
            print("에러 난 파일 명 > ", file_name)
            
    return result_list

In [23]:
train_dir_url = '.\\dataset\\HQ_512x512\\HQ_512x512' 
test_dir_url = '.\\dataset\\test_final_with_degrad\\test'

train_dataset = image_to_tensor(train_labels, train_dir_url)
print('*'*100)
test_dataset = image_to_tensor(test_labels, test_dir_url)

  4%|▎         | 73/2041 [00:00<00:05, 382.46it/s]

ERROR CODE >  224
에러 난 파일 명 >  5.jpg
ERROR CODE >  224
에러 난 파일 명 >  8.jpg
ERROR CODE >  224
에러 난 파일 명 >  17.jpg
ERROR CODE >  224
에러 난 파일 명 >  23.jpg
ERROR CODE >  224
에러 난 파일 명 >  26.jpg
ERROR CODE >  224
에러 난 파일 명 >  29.jpg
ERROR CODE >  224
에러 난 파일 명 >  30.jpg
ERROR CODE >  224
에러 난 파일 명 >  34.jpg
ERROR CODE >  224
에러 난 파일 명 >  35.jpg
ERROR CODE >  224
에러 난 파일 명 >  38.jpg
ERROR CODE >  224
에러 난 파일 명 >  39.jpg
ERROR CODE >  224
에러 난 파일 명 >  41.jpg
ERROR CODE >  224
에러 난 파일 명 >  51.jpg
ERROR CODE >  224
에러 난 파일 명 >  52.jpg
ERROR CODE >  224
에러 난 파일 명 >  53.jpg
ERROR CODE >  224
에러 난 파일 명 >  54.jpg
ERROR CODE >  224
에러 난 파일 명 >  55.jpg
ERROR CODE >  224
에러 난 파일 명 >  56.jpg
ERROR CODE >  224
에러 난 파일 명 >  57.jpg
ERROR CODE >  224
에러 난 파일 명 >  60.jpg
ERROR CODE >  224
에러 난 파일 명 >  63.jpg
ERROR CODE >  224
에러 난 파일 명 >  64.jpg
ERROR CODE >  224
에러 난 파일 명 >  65.jpg
ERROR CODE >  224
에러 난 파일 명 >  71.jpg
ERROR CODE >  224
에러 난 파일 명 >  72.jpg
ERROR CODE >  224
에러 난 파일 명 >  75.jpg
ERROR CODE >  

  9%|▊         | 176/2041 [00:00<00:03, 470.30it/s]

ERROR CODE >  224
에러 난 파일 명 >  365.jpg
ERROR CODE >  224
에러 난 파일 명 >  370.jpg
ERROR CODE >  224
에러 난 파일 명 >  371.jpg
ERROR CODE >  224
에러 난 파일 명 >  373.jpg
ERROR CODE >  224
에러 난 파일 명 >  374.jpg
ERROR CODE >  224
에러 난 파일 명 >  381.jpg
ERROR CODE >  224
에러 난 파일 명 >  384.jpg
ERROR CODE >  224
에러 난 파일 명 >  390.jpg
ERROR CODE >  224
에러 난 파일 명 >  392.jpg
ERROR CODE >  224
에러 난 파일 명 >  393.jpg
ERROR CODE >  224
에러 난 파일 명 >  420.jpg
ERROR CODE >  224
에러 난 파일 명 >  421.jpg
ERROR CODE >  224
에러 난 파일 명 >  422.jpg
ERROR CODE >  224
에러 난 파일 명 >  427.jpg
ERROR CODE >  224
에러 난 파일 명 >  428.jpg
ERROR CODE >  224
에러 난 파일 명 >  429.jpg
ERROR CODE >  224
에러 난 파일 명 >  430.jpg
ERROR CODE >  224
에러 난 파일 명 >  434.jpg
ERROR CODE >  224
에러 난 파일 명 >  436.jpg
ERROR CODE >  224
에러 난 파일 명 >  437.jpg
ERROR CODE >  224
에러 난 파일 명 >  441.jpg
ERROR CODE >  224
에러 난 파일 명 >  442.jpg
ERROR CODE >  224
에러 난 파일 명 >  445.jpg
ERROR CODE >  224
에러 난 파일 명 >  446.jpg
ERROR CODE >  224
에러 난 파일 명 >  464.jpg
ERROR CODE >  224
에러 난 파일

 13%|█▎        | 269/2041 [00:00<00:04, 403.57it/s]

...

ERROR CODE >  224
에러 난 파일 명 >  83653.jpg
ERROR CODE >  224
에러 난 파일 명 >  83654.jpg
ERROR CODE >  224
에러 난 파일 명 >  83655.jpg
ERROR CODE >  224
에러 난 파일 명 >  83657.jpg
ERROR CODE >  224
에러 난 파일 명 >  83658.jpg
ERROR CODE >  224
에러 난 파일 명 >  83659.jpg
ERROR CODE >  224
에러 난 파일 명 >  83660.jpg
ERROR CODE >  224
에러 난 파일 명 >  83661.jpg
ERROR CODE >  224
에러 난 파일 명 >  83662.jpg
ERROR CODE >  224
에러 난 파일 명 >  83740.jpg
ERROR CODE >  224
에러 난 파일 명 >  83752.jpg
ERROR CODE >  224
에러 난 파일 명 >  83757.jpg
ERROR CODE >  224
에러 난 파일 명 >  83778.jpg
ERROR CODE >  224
에러 난 파일 명 >  83780.jpg
ERROR CODE >  224
에러 난 파일 명 >  83782.jpg
ERROR CODE >  224
에러 난 파일 명 >  83783.jpg
ERROR CODE >  224
에러 난 파일 명 >  83785.jpg
ERROR CODE >  224
에러 난 파일 명 >  83883.jpg
ERROR CODE >  224
에러 난 파일 명 >  83884.jpg
ERROR CODE >  224
에러 난 파일 명 >  83891.jpg
ERROR CODE >  224
에러 난 파일 명 >  83892.jpg
ERROR CODE >  224
에러 난 파일 명 >  83990.jpg
ERROR CODE >  224
에러 난 파일 명 >  84035.jpg
ERROR CODE >  224
에러 난 파일 명 >  84037.jpg
ERROR CODE >  22

 72%|███████▏  | 58/81 [00:00<00:00, 568.86it/s]

ERROR CODE >  224
에러 난 파일 명 >  45.jpg
ERROR CODE >  224
에러 난 파일 명 >  85.jpg
ERROR CODE >  224
에러 난 파일 명 >  96.jpg
ERROR CODE >  224
에러 난 파일 명 >  112.jpg
ERROR CODE >  224
에러 난 파일 명 >  116.jpg
ERROR CODE >  224
에러 난 파일 명 >  127.jpg
ERROR CODE >  224
에러 난 파일 명 >  201.jpg
ERROR CODE >  224
에러 난 파일 명 >  252.jpg
ERROR CODE >  224
에러 난 파일 명 >  299.jpg
ERROR CODE >  224
에러 난 파일 명 >  312.jpg
ERROR CODE >  224
에러 난 파일 명 >  351.jpg
ERROR CODE >  224
에러 난 파일 명 >  425.jpg
ERROR CODE >  224
에러 난 파일 명 >  435.jpg
ERROR CODE >  224
에러 난 파일 명 >  448.jpg
ERROR CODE >  224
에러 난 파일 명 >  522.jpg
ERROR CODE >  224
에러 난 파일 명 >  610.jpg
ERROR CODE >  224
에러 난 파일 명 >  835.jpg
ERROR CODE >  224
에러 난 파일 명 >  884.jpg
ERROR CODE >  224
에러 난 파일 명 >  910.jpg
ERROR CODE >  224
에러 난 파일 명 >  1005.jpg
ERROR CODE >  224
에러 난 파일 명 >  1059.jpg
ERROR CODE >  224
에러 난 파일 명 >  1144.jpg
ERROR CODE >  224
에러 난 파일 명 >  1324.jpg
ERROR CODE >  224
에러 난 파일 명 >  1330.jpg
ERROR CODE >  224
에러 난 파일 명 >  1342.jpg
ERROR CODE >  224
에러 난

100%|██████████| 81/81 [00:00<00:00, 545.05it/s]

ERROR CODE >  224
에러 난 파일 명 >  52698.jpg
ERROR CODE >  224
에러 난 파일 명 >  53415.jpg
ERROR CODE >  224
에러 난 파일 명 >  53648.jpg
ERROR CODE >  224
에러 난 파일 명 >  53653.jpg
ERROR CODE >  224
에러 난 파일 명 >  83252.jpg
ERROR CODE >  224
에러 난 파일 명 >  84034.jpg


In [24]:
print("train_dataset 데이터 개수 : ", len(train_dataset))

lst = []
for i in range(len(train_dataset)):
    lst.append(train_dataset[i][1])
s_lst = pd.Series(lst)
print('train_dataset 라벨 개수', len(s_lst.unique()))

print('----------' * 3)

print("test_dataset 데이터 개수 : ", len(test_dataset))

lst = []
for i in range(len(test_dataset)):
    lst.append(test_dataset[i][1])
s_lst = pd.Series(lst)
print('test_dataset 라벨 개수', len(s_lst.unique()))

train_dataset 데이터 개수 :  0
train_dataset 라벨 개수 0
------------------------------
test_dataset 데이터 개수 :  0
test_dataset 라벨 개수 0


In [25]:
train_dataset[0][0].shape

IndexError: list index out of range

# 모델링

In [178]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [179]:
# DataLoader 만들기
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_laoder = DataLoader(dataset=test_dataset, batch_size=batch_size)

In [180]:
# VGG 모델은 여러가지 타입이 있으므로 타입에 맞춰 모델 생성하기 위해 layer를 거친 후 출력 kernel의 개수 미리 지정(M은 pooling layer)
cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

# A : VGG-11
# B : VGG-13
# D : VGG-16
# E : VGG-19

def make_layer(config): # 위에서 정의한 타입별로 모델 생성
    layers = []
    in_planes = 3 # input 개수
    for value in config:
        if value == "M": # Pooling layer일 때 
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        else: # Conv  layer일 때
            layers.append(nn.Conv2d(in_planes, value, kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            in_planes = value
    return nn.Sequential(*layers)

In [114]:
class VGG(nn.Module):
    def __init__(self, config, num_classes=1000, cifar=False):
        super(VGG, self).__init__()
        self.features = make_layer(config)
        
        # ImageNet
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)  
        )
        # CIFAR-10
        if cifar:
            self.classifier = nn.Sequential(
                nn.Dropout(0.5),
                nn.Linear(512, 512),
                nn.ReLU(True),
                nn.Dropout(0.5),
                nn.Linear(512, 512),
                nn.ReLU(True),
                nn.Linear(512, 10)  
            ) 
        
    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out,1)
        out = self.classifier(out)
        return out

In [115]:
# 지정한 VGG 타입 별 모델을 생성하는 함수 만들기
def VGG11(cifar=False):
    return VGG(config = cfg['A'], cifar = cifar)

def VGG13(cifar=False):
    return VGG(config = cfg['B'], cifar = cifar)

def VGG16(cifar=False):
    return VGG(config = cfg['D'], cifar = cifar)

def VGG19(cifar=False):
    return VGG(config = cfg['E'], cifar = cifar)

In [116]:
model = VGG16()

In [121]:
from torchsummary import summary

summary(model=model,
        input_size=(3, 224, 224),
        batch_size=batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 224, 224]           1,792
              ReLU-2         [32, 64, 224, 224]               0
            Conv2d-3         [32, 64, 224, 224]          36,928
              ReLU-4         [32, 64, 224, 224]               0
         MaxPool2d-5         [32, 64, 112, 112]               0
            Conv2d-6        [32, 128, 112, 112]          73,856
              ReLU-7        [32, 128, 112, 112]               0
            Conv2d-8        [32, 128, 112, 112]         147,584
              ReLU-9        [32, 128, 112, 112]               0
        MaxPool2d-10          [32, 128, 56, 56]               0
           Conv2d-11          [32, 256, 56, 56]         295,168
             ReLU-12          [32, 256, 56, 56]               0
           Conv2d-13          [32, 256, 56, 56]         590,080
             ReLU-14          [32, 256,